In [0]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.regression import LinearRegression

spark = SparkSession.builder.appName('lrex').getOrCreate()

In [0]:
cruise_data = spark.read.csv('/FileStore/tables/cruise_ship_info.csv', inferSchema=True, header=True)

In [0]:
cruise_data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [0]:
cruise_data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [0]:
cruise_data.groupBy('Cruise_line').count().show()

+-----------------+-----+
|      Cruise_line|count|
+-----------------+-----+
|            Costa|   11|
|              P&O|    6|
|           Cunard|    3|
|Regent_Seven_Seas|    5|
|              MSC|    8|
|         Carnival|   22|
|          Crystal|    2|
|           Orient|    1|
|         Princess|   17|
|        Silversea|    4|
|         Seabourn|    3|
| Holland_American|   14|
|         Windstar|    3|
|           Disney|    2|
|        Norwegian|   13|
|          Oceania|    3|
|          Azamara|    2|
|        Celebrity|   10|
|             Star|    6|
|  Royal_Caribbean|   23|
+-----------------+-----+



In [0]:
indexer = StringIndexer(inputCol='Cruise_line', outputCol='cruise_cat')
indexed = indexer.fit(cruise_data).transform(cruise_data)

In [0]:
indexed.head(1)

Out[42]: [Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55, cruise_cat=16.0)]

In [0]:
indexed.columns

Out[43]: ['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew',
 'cruise_cat']

In [0]:
assembler = VectorAssembler(inputCols=['Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density', 'cruise_cat'],
                            outputCol='features')

In [0]:
output = assembler.transform(indexed)

In [0]:
output.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)
 |-- cruise_cat: double (nullable = false)
 |-- features: vector (nullable = true)



In [0]:
output.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+--------------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|cruise_cat|            features|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+--------------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|[6.0,30.276999999...|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|[6.0,30.276999999...|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|       1.0|[26.0,47.262,14.8...|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|       1.0|[11.0,110.0,29.74...|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0| 

In [0]:
final_output_data = output.select('features', 'crew')

In [0]:
final_output_data.show()

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
|[26.0,47.262,14.8...| 6.7|
|[11.0,110.0,29.74...|19.1|
|[17.0,101.353,26....|10.0|
|[22.0,70.367,20.5...| 9.2|
|[15.0,70.367,20.5...| 9.2|
|[23.0,70.367,20.5...| 9.2|
|[19.0,70.367,20.5...| 9.2|
|[6.0,110.23899999...|11.5|
|[10.0,110.0,29.74...|11.6|
|[28.0,46.052,14.5...| 6.6|
|[18.0,70.367,20.5...| 9.2|
|[17.0,70.367,20.5...| 9.2|
|[11.0,86.0,21.24,...| 9.3|
|[8.0,110.0,29.74,...|11.6|
|[9.0,88.5,21.24,9...|10.3|
|[15.0,70.367,20.5...| 9.2|
|[12.0,88.5,21.24,...| 9.3|
|[20.0,70.367,20.5...| 9.2|
+--------------------+----+
only showing top 20 rows



In [0]:
train_data, test_data = final_output_data.randomSplit([0.7, 0.3])

In [0]:
lr = LinearRegression(labelCol='crew')

In [0]:
lr_model = lr.fit(train_data)

In [0]:
test_results = lr_model.evaluate(test_data)

In [0]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| -0.8906257431335005|
|  0.2640416141315747|
| -0.5431437755697779|
|  1.0944509834156761|
|-0.44034064529929395|
|0.031649962413541566|
|  -0.809438394818299|
|-0.28802627237784684|
|  0.7635385066991187|
|-0.07207204594977767|
|-0.08139261849711321|
| -0.3811813313388015|
|  0.3441120575940473|
|  0.3463420199350087|
| 0.17454524471867572|
| -0.8268328742440385|
| -0.6626087223744683|
| -0.4728440321350398|
| -0.7525758946457799|
| -0.6252321156402072|
+--------------------+
only showing top 20 rows



In [0]:
test_results.meanSquaredError

Out[56]: 0.3994212438578549

In [0]:
test_results.r2

Out[57]: 0.9533528222347724

In [0]:
final_output_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|              158|
|   mean|7.794177215189873|
| stddev|3.503486564627034|
|    min|             0.59|
|    max|             21.0|
+-------+-----------------+



In [0]:
unlabeled_data = test_data.select('features')

In [0]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[6.0,30.276999999...|
|[6.0,110.23899999...|
|[8.0,110.0,29.74,...|
|[9.0,113.0,26.74,...|
|[10.0,81.76899999...|
|[11.0,58.6,15.66,...|
|[11.0,85.0,18.48,...|
|[11.0,86.0,21.24,...|
|[12.0,88.5,21.24,...|
|[13.0,25.0,3.82,5...|
|[13.0,30.27699999...|
|[13.0,76.0,18.74,...|
|[14.0,30.27699999...|
|[14.0,30.27699999...|
|[14.0,33.0,4.9,5....|
|[14.0,77.104,20.0...|
|[16.0,78.491,24.3...|
|[17.0,70.0,20.76,...|
|[17.0,101.353,26....|
|[18.0,51.004,9.4,...|
+--------------------+
only showing top 20 rows



In [0]:
predictions = lr_model.transform(unlabeled_data)

In [0]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[6.0,30.276999999...|   4.4406257431335|
|[6.0,110.23899999...|11.235958385868425|
|[8.0,110.0,29.74,...|12.143143775569778|
|[9.0,113.0,26.74,...|11.285549016584325|
|[10.0,81.76899999...| 8.860340645299294|
|[11.0,58.6,15.66,...| 7.568350037586458|
|[11.0,85.0,18.48,...| 8.809438394818299|
|[11.0,86.0,21.24,...| 9.588026272377848|
|[12.0,88.5,21.24,...|  9.52646149330088|
|[13.0,25.0,3.82,5...| 3.022072045949778|
|[13.0,30.27699999...| 4.081392618497113|
|[13.0,76.0,18.74,...| 8.881181331338801|
|[14.0,30.27699999...|3.3858879424059527|
|[14.0,30.27699999...|3.3836579800649913|
|[14.0,33.0,4.9,5....|3.0654547552813245|
|[14.0,77.104,20.0...| 8.826832874244039|
|[16.0,78.491,24.3...| 8.312608722374469|
|[17.0,70.0,20.76,...|  7.67284403213504|
|[17.0,101.353,26....| 10.75257589464578|
|[18.0,51.004,9.4,...| 6.075232115640207|
+--------------------+------------